In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

# Distributed Computing
Sources: *Hadoop: The Definitive Guide, 4th Ed.*

## Hadoop
A reliable, scalable platform for storage and analysis. Its three primary components are a resource manager, a fault tolerant, distributed filesystem, and fault-tolerant distributed processing. 


## Hadoop File System (HDFS)
*A filesystem designed for storing very large files with streaming data access patterns, running on clusters of commodity hardware*  
"Streaming data access": Write-once, read-many-times.
- *Distributed filesystem*: a filesystem that manages the storage across a network of machines. A complex problem! Network of machines
- *Fault Tolerant*: nodes can go down without suffering data loss.
- *Optimized for delivering a high throughput of data*, maybe at expense of latency. HBase is better for low-latency access

**Exercise: Get acquainted with HDFS:** Your Digital Ocean box already has a Hadoop 2.x version installed. This means that the DO box, which is effectively a 16 GB RAM, quad-core computer in the cloud, can act like a single node running Hadoop. Hadoop can be run as a single Java process in non-distributed mode (i.e. standalone), or in pseudo-distributed mode where each Hadoop daemon (background process) runs in a separate process. That means on the box you can create and run Hadoop jobs that access the Hadoop distributed filesystem. This is useful for debugging.

At the command line of your DO box, try out a few HDFS commands:
```bash
$ hadoop fs -ls /
Found 3 items
drwxrwxrwt   - hdfs supergroup          0 2015-03-27 13:42 /tmp
drwxr-xr-x   - hdfs supergroup          0 2015-03-25 15:35 /user
drwxr-xr-x   - hdfs supergroup          0 2015-03-25 15:35 /var

$ hadoop fs -du -h /user

$ hadoop fs -mkdir gutenberg

$ hadoop fs -copyFromLocal small_data/gutenberg/* gutenberg

$ hadoop fs -cat gutenberg/pg20417.txt | head

# run this mapreduce
$ hadoop jar /usr/lib/hadoop-mapreduce/hadoop-mapreduce-examples-2.*.jar grep gutenberg output 'strange'

# note that there is an output dir
$ hadoop fs -ls

# it contains a _SUCCESS flag and part-r-00000
$ hadoop fs -ls output

# here's the contents
$ hadoop fs -cat output/part-r-00000 | head
```
A full list of available commands available [here](http://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html).

## YARN
**YARN** ("Yet Another Resource Negotiator"): a cluster resource management system introduced in Hadoop 2.0 (c. February 2013) which allows any distributed program (not just MapReduce) to run on data in a Hadoop cluster.
Critical for allowing new processing models in Hadoop, i.e. the flourishing of the Hadoop ecosystem. 

### The different processing patterns that work with Hadoop
- **Batch processing**: This is what Hadoop was designed to do.  You know what you want to do, write the job, and run it over large amounts of data.
- **Interactive SQL**: By dispensing with MapReduce and using a distributed query engine that uses dedicated “always on” daemons (like Impala) or container reuse (like *Hive* on Tez), it’s possible to achieve low-latency responses for SQL queries on Hadoop while still scaling up to large(-ish) dataset sizes.  
- **Iterative processing**: Many algorithms - such as those in machine learning - are iterative in nature, so it’s much more efficient to hold each intermediate working set in memory, compared to loading from disk on each iteration. The architecture of MapReduce does not allow this, but it’s straightforward with *Spark*, for example, and it enables a highly exploratory style of working with datasets.  
- **Stream processing**: Streaming systems like Storm, Spark Streaming, or Samza make it possible to run real-time, distributed computations on unbounded streams of data and emit results to Hadoop storage or external systems.  

## Writing MapReduce jobs using Hadoop streaming

### Word count, aka the "Hello World" of big data.

Let's walk through an example of writing a mapper function, a reducer function, running it locally (on your Digital Ocean box) to test it, then running it using the Hadoop streaming jar.  The canonical example of a program is *Hello World*.

In the `datacourse/scripts` dir you should see a `mapper.py` and a `reducer.py` script.  These scripts take input and output through [standard unix streams](https://en.wikipedia.org/wiki/Standard_streams), which are meant to be chained together.

### "Unix" mapreduce
The simplest way to test these scripts is through <a href='https://en.wikipedia.org/wiki/Pipeline_(Unix)'>unix pipes</a>.  From `datacourse`, run
``` bash
cat small_data/gutenberg/pg20417.txt \
    | python scripts/mapper.py \
    | sort -k1,1 \
    | python scripts/reducer.py \
    | grep "^[a-z]" \
    | more
```
This is the simplest single-node "implementation" of mapreduce.  It's great for testing.  Notice that the output is ordered alphabetically on the first key. 

### Streaming mapreduce
Unix pipes are constrained to only run on localhost.  When you want to run your code in a distributed manner, you will want to run them using hadoop streaming:
```bash
export STREAMING_JAR=/usr/lib/hadoop-mapreduce/hadoop-streaming-*.jar

hadoop jar $STREAMING_JAR \
    -mapper scripts/mapper.py \
    -reducer scripts/reducer.py \
    -input gutenberg \
    -output output_gutenberg
```
The mapper task converts the inputs into lines and feeds the lines into the stdin of the process. 
The output from stdout of the mapper process is converted into a key/value pair. By default, all characters up to the first tab character is the key and the rest of the line is the value. This is customizable.  Optional streaming command parameters:  
`-combiner, -inputformat, -outputformat, -numReduceTasks, -file` 

Optional generic command parameters:   
`-D` Configuration variables, e.g.
- ```-D mapred.reduce.tasks=0``` specifies a "map-only" job.
- ```-D stream.map.output.field.separator=.``` specifies '.' as the field separator instead of '\t'
- `-D output.compression.enabled=true` 
- `-D output.compression.codec=org.apache.hadoop.io.compress.BZip2Codec`

For full documentation, see: https://hadoop.apache.org/docs/r1.2.1/streaming.html

## MapReduce sequencing
![Mapreduce Process Structure from [Xiaocheng Zhang's blog](http://xiaochongzhang.me/blog/?p=338)](images/MapReduce_Work_Structure.png)

1. **Input**: the input is a large file on a distributed file system stored as an input system and read according to a user-speicifed `InputFormat`.  For word count, it is a large text file.
1. **Splitting**: an Input Reader splits the file into chunks of 64 MB or 128 MB.  We think of each chunk as consisting of a collection of records which can be represented as key-value pairs `(k0, v0)`.  We think of the records as being unordered but keyed.  The splitting is done so that each chunk is also on a single machine.  While there can be multiple chunkcs on a single machine, it is useful to abstract things and assume that each chunk resides on its own machine.
1. **Mapping**: the user specifies a `map` function, which iterates through the records within a chunk.  It outputs key-value pairs `(k1, v1)` (which are written to the local disk).  It can output multiple key-value pairs per input record or none at all and the key and value need not have the same type as the input.  In our case, each input record is a line in a file (e.g. `"Deer Bear River"`) and there is an output record consistoing of each word with the number one (e.g. `("Deer", 1)`), indicating that we saw the word `"Deer"` once.
1. **Combiner**: (skip this step the first time).  This runs a reduce right after mapping but before shuffling.  The **local aggregation** can reduce the amount of I/O in the shuffle phase, which is the most important bottleneck.
1. **Shuffle**: up until this point, all the computations have happened locally and no data has been exchanged between the servers.  During Shuffle, the key value pairs `(k1, v1)` are sent to a computer based on `k1` so that all pairs with the same key end up at the same computer (and are written to that computer's disk).  Think of `k1` as giving the *address* to which to send the data and `v1` as givnig the data payload to deliver.  Again, which we can have multiple keys mapping to the same physical machine, it is useful to abstract this and assume each key residing on its own machine.
1. **Reduce**: the user specifies a `reduce` function, which is given pairs `(k1, iter(v1))` where `iter(v1)` is an interator over the values.  An iterator is like a list except that there is no random access to elements and one can only iterate through its elements once in order.  Underneath the hood, all the data resides on disk and only the current record being iterated over is kept in memory.  The `reduce` function outputs key-value pairs `(k2, v2)` (which are written to the local disk).  Again, the number of values it outputs is independent of the number of inputs and they can be of different types.  In our case, it maps each `(k1, iter(v1))` to `(k1, sum(v1))`.
1. **Final Result**: The results at this point are written out to disk according ot a user-specified `OutputFormat`.

**Finer points**:
1. The records in mapreduce are meant to be unordered.
1. Mapreduce is fault tolerant, if one of the processes has an error, it will automatically be restarted.
1. You might think it's rediculous to write `("Car", 1)` twice for the input record `"Car Car River"`.  We could have written `("Car", 2)`.  How much memory woud it require to keep track of this?  In general, we want the amount of memory used to be constant in the number of input records - this is what makes mapreduce scalable.  That said, you should look into the `combiner` function.  The typical use case is to run reduce on all the values `(k1, iter(v1))` which happen to be on the same mapper computer as a speed up.  To find out more, look at the [Apache tutorial](https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html).  What does this save you, CPU time, RAM, or Network Load?
1. For more information, checkout [Apache](https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html).

### Job progression: 
1. Client applications submit jobs to the Job tracker.  
2. The JobTracker talks to the NameNode to determine the location of the data
3. The JobTracker locates TaskTracker nodes with available slots at or near the data
4. The JobTracker submits the work to the chosen TaskTracker nodes.
5. The TaskTracker nodes are monitored. If they do not submit heartbeat signals often enough, they are deemed to have failed and the work is scheduled on a different TaskTracker.
6. A TaskTracker will notify the JobTracker when a task fails. The JobTracker decides what to do then: it may resubmit the job elsewhere, it may mark that specific record as something to avoid, and it may may even blacklist the TaskTracker as unreliable.
7. When the work is completed, the JobTracker updates its status.

## The Hadoop ecosystem
![A Hadoop stack](images/Hadoop_ecosystem.png) 

## Comparing Hadoop with alternatives
- It's cheap: Hadoop itself is free; commodity clusters are inexpensive
- vs. Relational Database Management Systems
>  MapReduce is a good fit for problems that need to analyze the whole dataset in a batch fashion, particularly for ad hoc analysis. An RDBMS is good for point queries or updates, where the dataset has been indexed to deliver low-latency retrieval and update times of a relatively small amount of data. *MapReduce suits applications where the data is written once and read many times*, whereas a relational database is good for datasets that are continually updated.

- vs. Grid Computing or High Performance Computing
> Broadly, the approach in HPC is to distribute the work across a cluster of machines, which access a shared filesystem, hosted by a storage area network (SAN). *This works well for predominantly compute-intensive jobs*, but it becomes a problem when nodes need to access larger data volumes (hundreds of gigabytes, the point at which Hadoop really starts to shine), since the network band‐width is the bottleneck and compute nodes become idle.
Hadoop tries to co-locate the data with the compute nodes, so data access is fast because it is local. This feature, known as *data locality*, is at the heart of data processing in Hadoop and is the reason for its good performance. 

- Programming in Hadoop
> MPI gives great control to programmers, but it requires that they explicitly handle the mechanics of the data flow, exposed via **low-level** C routines and constructs such as sockets, as well as the higher-level algorithms for the analyses. Processing in Hadoop operates only at the **higher level**: the programmer thinks in terms of the data model (such as key-value pairs for MapReduce), while the data flow remains implicit.

### So, what's different about programming around big data?
- Performance considerations: get rid of straggler tasks (e.g. resulting from [skew](http://nuage.cs.washington.edu/pubs/opencirrus2011.pdf)). Ex: When using MapReduce, optimize the number of reducers; add a `combiner`
- Hardware configuring and tuning. 
- Monitoring the progress of your jobs
- Often: opaque error messages.


### If Hadoop is not working on DO, you might need to restart
```bash
sudo su root  # become the root user
echo JAVA_HOME=/usr/lib/jvm/java-8-oracle-amd64 >> /etc/environment
echo "export JAVA_HOME=/usr/lib/jvm/java-8-oracle-amd64" >> /etc/alternatives/hadoop-conf/hadoop-env.sh

# run command-D to get out of root mode

# Start HDFS
sudo service hadoop-hdfs-datanode start
sudo service hadoop-hdfs-namenode init
sudo service hadoop-hdfs-namenode start

sudo -u hdfs hadoop fs -mkdir /user/
sudo -u hdfs hadoop fs -mkdir /user/vagrant
sudo -u hdfs hadoop fs -chown vagrant /user/vagrant

# Check to make sure everything works now  
hadoop fs -put small_data/fha_by_tract.csv /user/vagrant
```

### Exit Tickets
1. Can/should the traveling salesman problem be solved using the MapReduce paradigm?
1. At which points in MapReduce are information transmitted over the network?
1. From the perspective of a worker node doing mapping, describe the format of data it sees as input and what it outputs. Same for reducing. 

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*